In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip /content/drive/MyDrive/train.zip -d /content/data

In [ ]:
import shutil
import os

def clean_mac_files(directory):

    macosx_dir = os.path.join(directory, '__MACOSX')
    if os.path.exists(macosx_dir):
        shutil.rmtree(macosx_dir)

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.startswith('._'):
                os.remove(os.path.join(root, file))

data_dir = '/content/data'
clean_mac_files(data_dir)

print("Cleaned up '__MACOSX' folder and '._' files.")


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


data_dir = '/content/data/'


train_dataset = datasets.ImageFolder(root=data_dir, transform=transform)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


for images, labels in train_loader:
    print(images.size())
    break


In [ ]:
from torchvision.datasets import ImageFolder
train_dataset = ImageFolder('/content/data', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)



In [ ]:
import torch.nn as nn
from torchvision import models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model=models.resnet18(pretrained=True)

num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

In [ ]:
!pip install torchsummary
from torchsummary import summary
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn as nn
from torchvision import models
from tqdm import tqdm


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
model = model.to(device)


for param in model.parameters():
    param.requires_grad = True


epochs = 5


for epoch in range(epochs):
    model.train()
    running_loss = 0.0


    with tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", unit="batch") as t:
        for images, labels in t:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()


            t.set_postfix(loss=loss.item())

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

    for name, param in model.named_parameters():
        if param.grad is not None:
            print(f"Gradients for {name} exist")
        else:
            print(f"No gradients for {name}")